In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import dask.dataframe as dd
import dask.array as da
import dask.bag as db

In [2]:
product_info = pd.read_csv("./data/Iowa_Liquor_Products.csv")

In [5]:
product_info.columns = [x.replace(" ", "") if x != "Bottle Volume (ml)" else "BottleVolumeML" for x in product_info.columns]

In [8]:
len(product_info.ItemNumber.unique())

10568

In [9]:
product_info.shape[0]

10568

In [ ]:
invoice_ddf = dd.read_parquet("./data/invoice/part.*.parquet")